In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import pathlib

BASE_DIR = pathlib.Path.cwd().parent.parent.resolve()
BASE_DIR

WindowsPath('C:/Users/Alejandro Feria/Desktop/Unal/8 semestre/pnl/SilkNews')

In [41]:
clean_df = pd.read_csv(f"{BASE_DIR}/src/data/clean_data/noticias_combinadas.csv")
clean_df['text'] = clean_df['text'].astype(str)
print(clean_df.shape)

muestra = 500
muestras_df = clean_df.sample(muestra, random_state=42)
print(muestras_df.shape)

clean_df.head()

(5568, 2)
(500, 2)


,text,label
0,en una rueda de prensa las autoridades dijeron...,True
1,una operacion en la que las autoridades irland...,True
2,el nuevo informe se basa en mas de entrevistas...,True
3,las pruebas hechas a varias de las principales...,True
4,michelle y john wylie estuvieron buscando dura...,True


# Crear BoW

In [5]:
import spacy
# uv add --dev pip
# uv run --with pip spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")
nlp

In [ ]:
def tokenize(text: str) -> list[str]:
  return [token.lemma_ for token in nlp(text) if not token.is_punct and not token.is_space]

muestras_df["tokens"] = muestras_df["text"].apply(tokenize)


In [43]:
muestras_df.head()

,text,label,tokens
1168,los socialistas reunen a cerca de cargos en un...,True,"[el, socialista, reunir, a, cerca, de, cargo, ..."
765,la candidata del pp a la comunidad de madrid i...,True,"[el, candidata, del, pp, a, el, comunidad, de,..."
465,con el objetivo de paliar las penurias de pers...,True,"[con, el, objetivo, de, paliar, el, penuria, d..."
1117,la desaceleracion economica y la normalizacion...,True,"[el, desaceleracion, economico, y, el, normali..."
5371,un fuerte miedo y temor experimento brayan per...,False,"[uno, fuerte, miedo, y, temor, experimento, br..."


## eliminar stopwords

In [46]:
def remove_stopwords(tokens: list[str]) -> list[str]:
  return [token for token in tokens if not nlp.vocab[token].is_stop]

muestras_df["sin_stopwords"] = muestras_df["tokens"].apply(remove_stopwords)
muestras_df.head()

,text,label,tokens,sin_stopwords
1168,los socialistas reunen a cerca de cargos en un...,True,"[el, socialista, reunir, a, cerca, de, cargo, ...","[socialista, reunir, cerca, cargo, jornada, tr..."
765,la candidata del pp a la comunidad de madrid i...,True,"[el, candidata, del, pp, a, el, comunidad, de,...","[candidata, pp, comunidad, madrid, isabel, dia..."
465,con el objetivo de paliar las penurias de pers...,True,"[con, el, objetivo, de, paliar, el, penuria, d...","[objetivo, paliar, penuria, persona, riesgo, e..."
1117,la desaceleracion economica y la normalizacion...,True,"[el, desaceleracion, economico, y, el, normali...","[desaceleracion, economico, normalizacion, mer..."
5371,un fuerte miedo y temor experimento brayan per...,False,"[uno, fuerte, miedo, y, temor, experimento, br...","[fuerte, miedo, temor, experimento, brayan, pe..."


In [69]:
countVectorizer_sin_stopwords = CountVectorizer()
x_bow_sin_stopwords = countVectorizer_sin_stopwords.fit_transform(muestras_df["sin_stopwords"].str.join(sep=" ").to_numpy())
print(countVectorizer_sin_stopwords.get_feature_names_out())
# print(x_bow_sin_stopwords.toarray())
print(print(countVectorizer_sin_stopwords.vocabulary_))

['aaiun' 'abajo' 'abalo' ... 'ﬁn' 'ﬁnal' 'ﬁrme']
{'socialista': 8998, 'reunir': 8441, 'cerca': 1714, 'cargo': 1566, 'jornada': 5566, 'trabajo': 9540, 'grupo': 4660, 'diputacion': 3156, 'alicante': 430, 'reunirar': 8442, 'municipal': 6576, 'alcalde': 371, 'portavoz': 7507, 'concejal': 2123, 'tratar': 9620, 'candidata': 1495, 'pp': 7561, 'comunidad': 2114, 'madrid': 5970, 'isabel': 5481, 'diaz': 3105, 'ayuso': 1038, 'denunciar': 2842, 'miercol': 6358, 'victimo': 9928, 'campana': 1462, 'machista': 5963, 'deleznable': 2798, 'aspirante': 879, 'gobernar': 4577, 'region': 8209, 'salir': 8652, 'defensa': 2768, 'vox': 10055, 'objetivo': 6798, 'paliar': 7036, 'penuria': 7242, 'persona': 7306, 'riesgo': 8484, 'exclusion': 3974, 'social': 8996, 'carita': 1572, 'castellon': 1615, 'lograr': 5916, 'programa': 7757, 'actividad': 138, 'llegar': 5881, 'beneficiario': 1165, 'poblacion': 7438, 'puesto': 7905, 'desaceleracion': 2884, 'economico': 3352, 'normalizacion': 6740, 'mercado': 6293, 'inmobiliario'

## Frecuencia menor a 3

In [77]:
import collections
# si la frencuencia es menos de 3 eliminar palabras
all_tokens = [token for sublist in muestras_df["tokens"] for token in sublist]
token_counts = collections.Counter(all_tokens)
tokens_to_remove = {token for token, count in token_counts.items() if count < 3}
muestras_df["frecuencia<3"] = muestras_df["tokens"].apply(lambda tokens: [token for token in tokens if token not in tokens_to_remove])

print(f"Tokens eliminados por frecuencia < 3: {len(tokens_to_remove)}")

Tokens eliminados por frecuencia < 3: 7157


In [78]:
muestras_df

,text,label,tokens,sin_stopwords,frecuencia<3
1168,los socialistas reunen a cerca de cargos en un...,True,"[el, socialista, reunir, a, cerca, de, cargo, ...","[socialista, reunir, cerca, cargo, jornada, tr...","[el, socialista, reunir, a, cerca, de, cargo, ..."
765,la candidata del pp a la comunidad de madrid i...,True,"[el, candidata, del, pp, a, el, comunidad, de,...","[candidata, pp, comunidad, madrid, isabel, dia...","[el, candidata, del, pp, a, el, comunidad, de,..."
465,con el objetivo de paliar las penurias de pers...,True,"[con, el, objetivo, de, paliar, el, penuria, d...","[objetivo, paliar, penuria, persona, riesgo, e...","[con, el, objetivo, de, el, de, persona, en, r..."
1117,la desaceleracion economica y la normalizacion...,True,"[el, desaceleracion, economico, y, el, normali...","[desaceleracion, economico, normalizacion, mer...","[el, economico, y, el, del, mercado, inmobilia..."
5371,un fuerte miedo y temor experimento brayan per...,False,"[uno, fuerte, miedo, y, temor, experimento, br...","[fuerte, miedo, temor, experimento, brayan, pe...","[uno, fuerte, miedo, y, experimento, brayan, p..."
...,...,...,...,...,...
4620,ante los acontecimientos ocurridos hoy en barc...,False,"[ante, el, acontecimiento, ocurrido, hoy, en, ...","[acontecimiento, ocurrido, barcelona, fanatism...","[ante, el, acontecimiento, ocurrido, hoy, en, ..."
471,cuatrocientas mil personas trabajaron a lo lar...,True,"[cuatrocienta, mil, persona, trabajar, a, él, ...","[cuatrocienta, mil, persona, trabajar, ano, di...","[mil, persona, trabajar, a, él, largo, de, sie..."
3039,«lo hare gratis…» afirma jose ignacio munilla ...,False,"[él, hare, gratis, afirmar, jo él, ignacio, mu...","[hare, gratis, afirmar, jo él, ignacio, munill...","[él, gratis, afirmar, jo él, ignacio, obispo, ..."
2577,menores migrantes en altafulla g battista los ...,True,"[menor, migrant, en, altafulla, g, battista, e...","[menor, migrant, altafulla, g, battista, mosso...","[menor, migrant, en, g, el, mossos, desquadra,..."


In [79]:
# si la frencuencia es menos de 3 eliminar
countVectorizer_frecuencia = CountVectorizer()
x_bow_frecuencia = countVectorizer_frecuencia.fit_transform(muestras_df["frecuencia<3"].str.join(sep=" ").to_numpy())
print(countVectorizer_frecuencia.get_feature_names_out())
print(print(countVectorizer_frecuencia.vocabulary_))

['abajo' 'abalo' 'abanderado' ... 'zodiaco' 'zona' 'él']
{'el': 1160, 'socialista': 3022, 'reunir': 2831, 'cerca': 570, 'de': 902, 'cargo': 517, 'en': 1192, 'uno': 3278, 'jornada': 1831, 'trabajo': 3207, 'grupo': 1570, 'alicante': 156, 'hoy': 1641, 'municipal': 2175, 'alcalde': 134, 'portavoz': 2507, 'concejal': 709, 'que': 2670, 'él': 3412, 'tratar': 3232, 'candidata': 489, 'del': 946, 'pp': 2525, 'comunidad': 703, 'madrid': 1962, 'isabel': 1810, 'diaz': 1032, 'haber': 1580, 'denunciar': 968, 'este': 1313, 'miercol': 2099, 'ser': 2965, 'campana': 473, 'mas': 2026, 'machista': 1960, 'hacer': 1593, 'contra': 781, 'gobernar': 1544, 'region': 2763, 'vez': 3340, 'salir': 2898, 'defensa': 939, 'vox': 3387, 'su': 3059, 'con': 707, 'objetivo': 2265, 'persona': 2441, 'riesgo': 2844, 'social': 3021, 'carita': 519, 'castellon': 534, 'lograr': 1945, 'programa': 2603, 'actividad': 48, 'llegar': 1934, 'durante': 1123, 'beneficiario': 385, 'poblacion': 2478, 'puesto': 2660, 'economico': 1130, 'merca